<a href="https://colab.research.google.com/github/nisaodabas/CNG483-Project2/blob/master/CNG483_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model  
from keras.utils import np_utils
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from google.colab import output
from google.colab import drive

drive.mount('/content/drive')

Using TensorFlow backend.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 1

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
#sess = tf.compat.v1.Session(graph=tf.get_default_graph(), config=session_conf)
#K.set_session(sess)
# for later versions:
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)

In [0]:
def createDataSet(path):
    df = pd.read_csv(path, sep='\n')
    data = np.array(df)

    #print(data)
    numAttr = 0
    rows = []
    for row in data:        
        found = row[0].find("@ATTRIBUTE")
        if(found != -1):
            numAttr += 1
        else:
            # ignore @ATTRIBUTE rows
            rows.append(row[0].split(","))

    # delete @DATA row
    rows = np.array(rows[1:])

    labels = rows[:, -1].astype('int')
    
    labels = np.array([x - 1 for x in labels])

    if(numAttr == 6):
        features = rows[:, :numAttr-1].astype('float')
    else:
        features = rows[:, :numAttr-1].astype('int')
        # define feature selection
        #fs = SelectKBest(score_func=f_classif, k=4000)
        # apply feature selection
        #features = fs.fit_transform(features, labels)
        



    '''
    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(labels)
    encoded_Y = encoder.transform(labels)
    # convert integers to dummy variables (i.e. one hot encoded)
    labels = np_utils.to_categorical(encoded_Y)
    print(labels)
    '''
    return features, labels

In [0]:
def model(X, y, dim):
    
    # define model
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, shuffle=True, stratify=y, random_state=1)
    
    
    model = Sequential()
    model.add(Dense(15, activation='relu', kernel_initializer='he_normal', input_dim=dim))

    model.add(Dense(12, activation='relu', kernel_initializer='he_normal'))
    
    model.add(Dropout(0.4))
    
    model.add(Dense(12, activation='relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.4))
    
    model.add(Dense(9, activation='relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.4))

    model.add(Dense(6, activation='relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.4))
    
    model.add(Dense(3, activation='softmax'))
    # compile the model
    opt = RMSprop(learning_rate=0.001, rho=0.9, momentum=0.0)
    #model.compile(optimizer=opt, loss='binary_crossentropy')
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['acc'])
    
    # simple early stopping
    #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
    #mc = ModelCheckpoint('/content/drive/My Drive/CNG483-Project 2/best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    
    numEpochs = 3000
    batches = 256
    # fit model , callbacks=[es]
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=numEpochs, batch_size=batches, verbose=1)

    # evaluate the model
    _, train_acc = model.evaluate(X_train, y_train, batch_size=batches, verbose=0)
    _, test_acc = model.evaluate(X_val, y_val, batch_size=batches, verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
    
    
    
    
    #history = model.fit(X_train, y_train, epochs=numEpochs, batch_size=128, validation_data=(X_val, y_val))

    #print(history.history.keys())

    loss_train = history.history['loss']
    loss_val = history.history['val_loss']
    epochs = range(1,numEpochs+1)
    plt.plot(epochs, loss_train, 'g', label='Training loss')
    plt.plot(epochs, loss_val, 'b', label='validation loss')
    plt.title('Training and Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    acc_train = history.history['acc']
    acc_val = history.history['val_acc']
    epochs = range(1,numEpochs+1)
    plt.plot(epochs, acc_train, 'g', label='Training accuracy')
    plt.plot(epochs, acc_val, 'b', label='validation accuracy')
    plt.title('Training and Validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
    #return model

In [0]:
if __name__ == "__main__":
    geometic_test_path = "/content/drive/My Drive/CNG483-Project 2/IrisGeometicFeatures_TestingSet.txt"
    geometic_train_path = "/content/drive/My Drive/CNG483-Project 2/IrisGeometicFeatures_TrainingSet.txt"

    texture_test_path = "/content/drive/My Drive/CNG483-Project 2/IrisTextureFeatures_TestingSet.txt"
    texture_train_path = "/content/drive/My Drive/CNG483-Project 2/IrisTextureFeatures_TrainingSet.txt"


    geo_x_train, geo_y_train = createDataSet(geometic_train_path)
    geo_x_test, geo_y_test = createDataSet(geometic_test_path)

    txtr_x_train, txtr_y_train = createDataSet(texture_train_path)
    txtr_x_test, txtr_y_test = createDataSet(texture_test_path)


In [0]:
    all_x_train = np.concatenate((txtr_x_train, geo_x_train), axis=1)
    

In [0]:
    #model(txtr_x_train, txtr_y_train, 9600)
    #model(geo_x_train, geo_y_train, 5)
    model(all_x_train, txtr_y_train, 9605)

    #conv1d(txtr_x_train, txtr_y_train, 9600)
    '''
    estimator = KerasClassifier(build_fn=model, epochs=2000, batch_size=128, verbose=0)
    kfold = KFold(n_splits=5, shuffle=True, random_state=0)
    results = cross_val_score(estimator, txtr_x_train, txtr_y_train, cv=kfold, scoring='accuracy')

    print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    '''
    '''
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    '''

Epoch 1/3000
5/5 [==============================] - 0s 51ms/step - loss: 80.4049 - acc: 0.4509 - val_loss: 8.2019 - val_acc: 0.5739
Epoch 2/3000
5/5 [==============================] - 0s 18ms/step - loss: 34.3254 - acc: 0.4616 - val_loss: 4.1390 - val_acc: 0.5739
Epoch 3/3000
5/5 [==============================] - 0s 18ms/step - loss: 16.7712 - acc: 0.4752 - val_loss: 3.8310 - val_acc: 0.5739
Epoch 4/3000
5/5 [==============================] - 0s 18ms/step - loss: 11.5526 - acc: 0.4752 - val_loss: 4.8481 - val_acc: 0.5739
Epoch 5/3000
5/5 [==============================] - 0s 18ms/step - loss: 10.3878 - acc: 0.4606 - val_loss: 3.0096 - val_acc: 0.5739
Epoch 6/3000
5/5 [==============================] - 0s 19ms/step - loss: 6.5522 - acc: 0.4616 - val_loss: 1.3699 - val_acc: 0.5739
Epoch 7/3000
5/5 [==============================] - 0s 18ms/step - loss: 2.2595 - acc: 0.4286 - val_loss: 1.0847 - val_acc: 0.4000
Epoch 8/3000
5/5 [==============================] - 0s 18ms/step - loss: 1.135

In [0]:
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')